In [ ]:
import requests
import pandas as pd


team_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 33, 34]


all_team_dfs = []


for team_id in team_ids:
    
    api_endpoint_info = f'https://site.api.espn.com/apis/site/v2/sports/football/nfl/teams/{team_id}'

    response_info = requests.get(api_endpoint_info)

    if response_info.status_code == 200:
        
        json_data_info = response_info.json()

        team_info = {
            'displayName': json_data_info['team']['displayName'],
            'location': json_data_info['team']['location'],
            'abbreviation': json_data_info['team']['abbreviation'],
            'color': json_data_info['team']['color']
        }

        team_df_info = pd.DataFrame([team_info])

        api_endpoint_stats = f'https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/2021/types/2/teams/{team_id}/statistics'

        response_stats = requests.get(api_endpoint_stats)

        if response_stats.status_code == 200:
        
            json_data_stats = response_stats.json()

            team_stats = {'Team ID': team_id}
            for category in json_data_stats['splits']['categories']:
                category_name = category['displayName']
                team_stats[category_name] = {}
                
                for stat in category['stats']:               
                    rank_value = stat.get('rank', 'N/A')  
                    stat_name_display = stat['displayName'].replace(" ", "")  
                    stat_name = f"{category_name}_{stat_name_display}"  
                    team_stats[f"{category_name}_{stat_name_display}_StatName"] = stat['displayName']
                    team_stats[f"{category_name}_{stat_name_display}_StatDescription"] = stat['description']
                    team_stats[f"{category_name}_{stat_name_display}_StatValue"] = stat['value']
                    team_stats[f"{category_name}_{stat_name_display}_StatRank"] = rank_value
                    
            
            team_df_stats = pd.json_normalize(team_stats)

            team_df_combined = pd.concat([team_df_info, team_df_stats], axis=1)

            all_team_dfs.append(team_df_combined)


final_combined_df = pd.concat(all_team_dfs, ignore_index=True)

team_info_columns = ['Team ID', 'displayName', 'location', 'abbreviation', 'color']
final_combined_df = final_combined_df[team_info_columns + [col for col in final_combined_df.columns if col not in team_info_columns]]

final_combined_df = pd.concat(all_team_dfs, ignore_index=True)

column_name_mapping = {
    'displayName': 'TeamName',
    'location': 'TeamLocation',
    'abbreviation': 'TeamAbbreviation',
    'color': 'TeamColor'
}

final_combined_df = final_combined_df.rename(columns=column_name_mapping)

final_combined_df.index = final_combined_df.index + 1

final_combined_df = final_combined_df.drop('Team ID', axis=1)

final_combined_df.to_csv("NFLTeamStats_'21-'22.csv", index=False)

""" Display the final combined DataFrame

final_combined_df.head()"""



